In [1]:
import folium
import pandas as pd
import numpy as np

# Import folium DivIcon plugin
from folium.features import DivIcon

# Import folium MousePosition plugin
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster, MousePosition

In [2]:
# Setting this option will print all collumns of a dataframe
pd.set_option("display.max_columns", None)

# Euro 2024 Tournament Data
df = pd.read_csv("data/2024.csv")

# convert to datetime objects
df["date_time"] = pd.to_datetime(df["date_time"])

In [3]:
df.head(10)

,id_match,home_team,away_team,home_team_code,away_team_code,home_score,away_score,home_penalty,away_penalty,home_score_total,away_score_total,winner,winner_reason,year,date,date_time,utc_offset_hours,group_name,matchday_name,condition_humidity,condition_pitch,condition_temperature,condition_weather,condition_wind_speed,status,type,round,round_mode,match_attendance,stadium_id,stadium_country_code,stadium_capacity,stadium_latitude,stadium_longitude,stadium_pitch_length,stadium_pitch_width,goals,penalties_missed,penalties,red_cards,game_referees,stadium_city,stadium_name,stadium_name_media,stadium_name_official,stadium_name_event,stadium_name_sponsor,home_lineups,away_lineups,home_coaches,away_coaches,events
0,2036211,Spain,England,ESP,ENG,2.0,1.0,NaN,NaN,2.0,1.0,Spain,WIN_REGULAR,2024,2024-07-14,2024-07-14 19:00:00+00:00,2.0,NaN,MD7,NaN,NaN,NaN,NaN,NaN,FINISHED,SINGLE,FINAL,FINAL,65600.0,62875,GER,74244.0,52.514714,13.239397,105.0,68.0,"[{'phase': 'SECOND_HALF', 'time': {'minute': 4...",NaN,NaN,NaN,"[{'name': 'Szymon Marciniak', 'role': 'FOURTH_...",Berlin,Olympiastadion,Olympiastadion,Olympiastadion,Olympiastadion,Olympiastadion,"[{'country_code': 'ESP', 'name': 'David Raya',...","[{'country_code': 'ENG', 'name': 'Aaron Ramsda...","[{'country_code': 'ESP', 'country': 'Spain', '...","[{'country_code': 'ENG', 'country': 'England',...",[{'id': 'c9451938-1653-4ed3-82af-3c3b641c9933'...
1,2036210,Netherlands,England,NED,ENG,1.0,2.0,NaN,NaN,1.0,2.0,England,WIN_REGULAR,2024,2024-07-10,2024-07-10 19:00:00+00:00,2.0,NaN,MD6,82.0,EXCELLENT,20.0,CLOUDY_NIGHT,11.0,FINISHED,SINGLE,SEMIFINAL,KNOCK_OUT,60926.0,57798,GER,81365.0,51.492589,7.451772,105.0,68.0,"[{'phase': 'FIRST_HALF', 'time': {'minute': 7,...",NaN,NaN,NaN,"[{'name': 'Marco Achmüller', 'role': 'ASSISTAN...",Dortmund,BVB Stadion Dortmund,BVB Stadion Dortmund,BVB Stadion Dortmund,BVB Stadion Dortmund,Signal Iduna Park,"[{'country_code': 'NED', 'name': 'Justin Bijlo...","[{'country_code': 'ENG', 'name': 'Aaron Ramsda...","[{'country_code': 'NED', 'country': 'Netherlan...","[{'country_code': 'ENG', 'country': 'England',...",[{'id': 'f5eaa274-731c-450c-b463-fa39b3159859'...
2,2036209,Spain,France,ESP,FRA,2.0,1.0,NaN,NaN,2.0,1.0,Spain,WIN_REGULAR,2024,2024-07-09,2024-07-09 19:00:00+00:00,2.0,NaN,MD6,NaN,NaN,NaN,NaN,NaN,FINISHED,SINGLE,SEMIFINAL,KNOCK_OUT,62042.0,85441,GER,75000.0,48.218794,11.624731,105.0,68.0,"[{'phase': 'FIRST_HALF', 'time': {'minute': 9,...",NaN,NaN,NaN,"[{'name': 'Paolo Valeri', 'role': 'ASSISTANT_V...",Munich,Munich Football Arena,Football Arena Munich,Fußball Arena München,Munich Football Arena,Allianz Arena,"[{'country_code': 'ESP', 'name': 'David Raya',...","[{'country_code': 'FRA', 'name': 'Brice Samba'...","[{'country_code': 'ESP', 'country': 'Spain', '...","[{'country_code': 'FRA', 'country': 'France', ...",[{'id': '1c23408d-cff7-4b73-984e-b0fe138a934e'...
3,2036207,Netherlands,Türkiye,NED,TUR,2.0,1.0,NaN,NaN,2.0,1.0,Netherlands,WIN_REGULAR,2024,2024-07-06,2024-07-06 19:00:00+00:00,2.0,NaN,MD5,NaN,NaN,NaN,NaN,NaN,FINISHED,SINGLE,QUARTER_FINALS,KNOCK_OUT,70091.0,62875,GER,74244.0,52.514714,13.239397,105.0,68.0,"[{'phase': 'FIRST_HALF', 'time': {'minute': 35...",NaN,NaN,"[{'phase': 'SECOND_HALF', 'time': {'injuryMinu...","[{'name': 'Clément Turpin', 'role': 'REFEREE',...",Berlin,Olympiastadion,Olympiastadion,Olympiastadion,Olympiastadion,Olympiastadion,"[{'country_code': 'NED', 'name': 'Justin Bijlo...","[{'country_code': 'TUR', 'name': 'Altay Bayınd...","[{'country_code': 'NED', 'country': 'Netherlan...","[{'country_code': 'ITA', 'country': 'Italy', '...",[{'id': 'c127c084-8277-4dfa-8c97-259c9333c770'...
4,2036208,England,Switzerland,ENG,SUI,1.0,1.0,5.0,3.0,1.0,1.0,England,WIN_ON_PENALTIES,2024,2024-07-06,2024-07-06 16:00:00+00:00,2.0,NaN,MD5,NaN,NaN,NaN,NaN,NaN,FINISHED,SINGLE,QUARTER_FINALS,KNOCK_OUT,46907.0,1100174,GER,44559.0,51.260304,6.732731,105.0,68.0,"[{'phase': 'SECOND_HALF', 'time': {'minute': 7...",NaN,"[{'phase': 'PENALTY', 'time': {}, 'internat

In [4]:
# sort dataframe entries by index, beginning with the highest index
df.sort_index(ascending=False, inplace=True)

# reset index, so that new indexing starts with the first match of the Euro 2024 Tournament
df.reset_index(drop=True, inplace=True)

df["stadium_capacity"] = df["stadium_capacity"].astype(int)
    

In [5]:
df.columns

Index(['id_match', 'home_team', 'away_team', 'home_team_code',
       'away_team_code', 'home_score', 'away_score', 'home_penalty',
       'away_penalty', 'home_score_total', 'away_score_total', 'winner',
       'winner_reason', 'year', 'date', 'date_time', 'utc_offset_hours',
       'group_name', 'matchday_name', 'condition_humidity', 'condition_pitch',
       'condition_temperature', 'condition_weather', 'condition_wind_speed',
       'status', 'type', 'round', 'round_mode', 'match_attendance',
       'stadium_id', 'stadium_country_code', 'stadium_capacity',
       'stadium_latitude', 'stadium_longitude', 'stadium_pitch_length',
       'stadium_pitch_width', 'goals', 'penalties_missed', 'penalties',
       'red_cards', 'game_referees', 'stadium_city', 'stadium_name',
       'stadium_name_media', 'stadium_name_official', 'stadium_name_event',
       'stadium_name_sponsor', 'home_lineups', 'away_lineups', 'home_coaches',
       'away_coaches', 'events'],
      dtype='object')

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
df_folium = df[
    [
        "stadium_name_official",
        "stadium_latitude",
        "stadium_longitude",
        "stadium_capacity",
        "matchday_name",
        "date_time",
        "home_team",
        "away_team",
        "home_score_total",
        "away_score_total",
        "home_penalty",
        "away_penalty",
        "id_match",
        "winner_reason",
        "status"
    ]
]
df_stadiums = df_folium.groupby(["stadium_name_official"], as_index=False).first()
df_stadiums = df_stadiums[
    [
        "stadium_name_official",
        "stadium_latitude",
        "stadium_longitude",
        "stadium_capacity",
    ]
]

In [7]:
df_stadiums

,stadium_name_official,stadium_latitude,stadium_longitude,stadium_capacity
0,Arena AufSchalke,51.554542,7.067569,54740
1,Arena Stuttgart,48.792269,9.232008,54697
2,BVB Stadion Dortmund,51.492589,7.451772,81365
3,Düsseldorf Arena,51.260304,6.732731,44559
4,Frankfurt Stadion,50.068586,8.645456,49474
5,Fußball Arena München,48.218794,11.624731,75000
6,Olympiastadion,52.514714,13.239397,74244
7,RB Arena,51.345733,12.348181,42146
8,Stadion Köln,50.933506,6.874994,45134
9,Volksparkstadion,53.587156,9.898553,51831


In [8]:
# Initialize the map
map_stadiums = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in df_stadiums.iterrows():
    circle = folium.Circle(
        [row["stadium_latitude"], row["stadium_longitude"]],
        radius=2000,
        color="#d35400",
        fill=True,
    ).add_child(
        folium.Popup(
            f'Official Stadium Name:<br>{row["stadium_name_official"]}<br><br>Stadium Capacity:<br>{row["stadium_capacity"]}',
            max_width=200,
        )
    )

    marker = folium.map.Marker(
        [row["stadium_latitude"], row["stadium_longitude"]],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row["stadium_name_official"],
        ),
    )
    map_stadiums.add_child(circle)
    map_stadiums.add_child(marker)

map_stadiums

In [9]:
marker_cluster = MarkerCluster()

In [10]:
def assign_marker_color(matchday):
    if matchday in ["MD1", "MD2", "MD3"]:
        return "lightblue"
    elif matchday == "MD4":
        return "cadetblue"
    elif matchday == "MD5":
        return "blue"
    elif matchday == "MD6":
        return "darkblue"
    else:
        return "darkpurple"


def assign_correct_matchday_names(matchday):
    matchday_mapping = {
        "MD4": "Round of 16",
        "MD5": "Quarter-finals",
        "MD6": "Semi-finals",
        "MD7": "Final",
    }
    return matchday_mapping.get(matchday, matchday)

def get_end_result(id_to_find):
    row = df_folium.loc[df_folium['id_match'] == id_to_find]
    if row['winner_reason'].values[0] == "WIN_ON_PENALTIES":
        return f'{int(row['home_penalty'].values[0])} : {int(row['away_penalty'].values[0])}<br>(after Penalties)'
    elif row['status'].values[0] == "FINISHED":
        return f'{int(row['home_score_total'].values[0])} : {int(row['away_score_total'].values[0])}'
    else:
        return f'{np.nan} : {np.nan}'


df_folium = df_folium.copy()
df_folium.loc[:, "marker_color"] = df_folium["matchday_name"].apply(assign_marker_color)
df_folium.loc[:, "matchday_name"] = df_folium["matchday_name"].apply(
    assign_correct_matchday_names
)

df_folium.head()

,stadium_name_official,stadium_latitude,stadium_longitude,stadium_capacity,matchday_name,date_time,home_team,away_team,home_score_total,away_score_total,home_penalty,away_penalty,id_match,winner_reason,status,marker_color
0,Fußball Arena München,48.218794,11.624731,75000,MD1,2024-06-14 19:00:00+00:00,Germany,Scotland,5.0,1.0,NaN,NaN,2036161,WIN_REGULAR,FINISHED,lightblue
1,Stadion Köln,50.933506,6.874994,45134,MD1,2024-06-15 13:00:00+00:00,Hungary,Switzerland,1.0,3.0,NaN,NaN,2036162,WIN_REGULAR,FINISHED,lightblue
2,Olympiastadion,52.514714,13.239397,74244,MD1,2024-06-15 16:00:00+00:00,Spain,Croatia,3.0,0.0,NaN,NaN,2036163,WIN_REGULAR,FINISHED,lightblue
3,BVB Stadion Dortmund,51.492589,7.451772,81365,MD1,2024-06-15 19:00:00+00:00,Italy,Albania,2.0,1.0,NaN,NaN,2036164,WIN_REGULAR,FINISHED,lightblue
4,Volksparkstadion,53.587156,9.898553,51831,MD1,2024-06-16 13:00:00+00:00,Poland,Netherlands,1.0,2.0,NaN,NaN,2036167,WIN_REGULAR,FINISHED,lightblue


In [11]:
map_stadiums.add_child(marker_cluster)

for index, record in df_folium.iterrows():
    marker = folium.Marker(
        [record["stadium_latitude"], record["stadium_longitude"]],
        icon=folium.Icon(color="white", icon_color=record["marker_color"], shadow=True),
    )
    popup_text = f'Date:<br>{record["date_time"].date()}<br><br>Time:<br>{record["date_time"].time()}<br><br>Matchday:<br>{record["matchday_name"]}<br><br>Result:<br>{record["home_team"]} : {record["away_team"]}<br>{get_end_result(record["id_match"])}'
    marker.add_child(folium.Popup(popup_text, max_width=300))
    marker_cluster.add_child(marker)

map_stadiums

In [12]:
# Create
teams = df[["home_team", "away_team"]].values
teams = np.unique(teams.flatten()).tolist()
print(teams)
print("\nNumber of list values:", len(teams))
print("Number of participating teams:", 24)
print("List contains only unique values:", len(teams) == len(set(teams)))

['Albania', 'Austria', 'Belgium', 'Croatia', 'Czechia', 'Denmark', 'England', 'France', 'Georgia', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Scotland', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Switzerland', 'Türkiye', 'Ukraine']

Number of list values: 24
Number of participating teams: 24
List contains only unique values: True
